In [85]:
%matplotlib inline
import pyspeckit
import numpy as np
from astropy import units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.convolution import convolve, Box1DKernel
from c_normalize import c_normalize
import ipywidgets as widgets
import glob
from IPython.display import display
import pandas as pd


plt.style.use("seaborn-darkgrid")

# to get widgets to work in jupyter lab: jupyter labextension install @jupyter-widgets/jupyterlab-manager
# need to install nodejs as well

## Visualizing Fit Performance on the Master Line List
With the Fe lines, fit performance was assessed in a manual fashion. With more than double the lines for the master list, it's worthwhile to come up with a more elegant solution for assessing fit performance. It will still need to be manual, but rather than looping through all of the spectra, I'd like to try building a jupyter widget that plugs into our dataset and allows for more interactive inspection of the fit

In [88]:
# Widgets


files = glob.glob("../pydata/ew_known/inputs/*wavsoln.fits")
files = [file.split("/")[-1]for file in files]

f_drop = widgets.Dropdown(
    options=files,
    value='col110_1134red_oned_25jan14_wavsoln.fits',
    description='File:',
    disabled=False,
)
line_list_df = pd.read_csv("master_line_list.csv")
line_list = [f'{species} {np.round(wav,3)}' for species, wav in zip(line_list_df['Species'],line_list_df['Wavelength'])]
l_drop = widgets.Dropdown(
    options=line_list,
    value='C I 5052.167',
    description='Line:',
    disabled=False,
)

mf_drop = widgets.Dropdown(
    options=['1','2','3'],
    value='2',
    description='Num. fits:',
    disabled=False,
)

left_reg = widgets.Text(
    value='0.3',
    placeholder='Type something',
    description=f'Left Region',
    disabled=False
)

right_reg = widgets.Text(
    value='0.3',
    placeholder='Type something',
    description=f'Right Region',
    disabled=False
)

fit_text = []
for i in range(3):
    fit_group = []
    fit_group.append(widgets.Text(
    value='-0.3',
    placeholder='Type something',
    description=f'Amp {i+1}:',
    disabled=False
))
    
    fit_group.append(widgets.Text(
    value='0',
    placeholder='Type something',
    description=f'Offset {i+1}:',
    disabled=False
))

    
    fit_group.append(widgets.Text(
    value='0.15',
    placeholder='Type something',
    description=f'Width {i+1}:',
    disabled=False
))
    fit_text.append(fit_group)

#display(f_drop, mf_drop)
#for group in fit_text:
#    display(*group)

def fitter(file, line, n_fits, left_reg, right_reg,fit1_a, fit1_o, fit1_w, fit2_a, fit2_o, fit2_w, fit3_a, fit3_o, fit3_w):
    print(file, n_fits, fit2_o)
    
def on_nfit_change(nfit):
    with out:
        fit_text = []
        for i in range(int(nfit)):
            fit_group = []
            fit_group.append(widgets.Text(
            value='-0.3',
            placeholder='Type something',
            description=f'Amp {i+1}:',
            disabled=False
        ))
    
            fit_group.append(widgets.Text(
            value='0',
            placeholder='Type something',
            description=f'Offset {i+1}:',
            disabled=False
        ))

    
            fit_group.append(widgets.Text(
            value='0.15',
            placeholder='Type something',
            description=f'Width {i+1}:',
            disabled=False
        ))
            fit_text.append(fit_group)

        fit_hbox = widgets.HBox([widgets.VBox(group) for group in fit_text])
        vbox = widgets.VBox([f_drop,mf_drop, fit_hbox, out])           
    
out = widgets.interactive_output(fitter,{'file': f_drop, 'line': l_drop, 'n_fits': mf_drop, 'left_reg': left_reg, 
                                         'right_reg': right_reg,
                                         'fit1_a':fit_text[0][0], 'fit1_o':fit_text[0][1], 'fit1_w':fit_text[0][2],
                                         'fit2_a':fit_text[1][0], 'fit2_o':fit_text[1][1], 'fit2_w':fit_text[1][2],
                                         'fit3_a':fit_text[2][0], 'fit3_o':fit_text[2][1], 'fit3_w':fit_text[2][2]})

option_hbox = widgets.HBox([f_drop, l_drop])
fit_hbox = widgets.HBox([widgets.VBox(group) for group in fit_text])
vbox = widgets.VBox([option_hbox,out,mf_drop,fit_hbox])
vbox